In [1]:
import pandas as pd
import numpy as np
from WindPy import w
import io
import matplotlib.pyplot as plt
import seaborn as sns
import os
import requests
%matplotlib inline

In [2]:
df = pd.read_excel('D:/Data/index-component/HSCAI.HI.xlsx')

In [3]:
df

,i_weight,wind_code,sec_name
0,11.614679,601318 CH Equity,NaN
1,9.228530,600519 CH Equity,NaN
2,6.259465,000333 CH Equity,NaN
3,5.590536,000651 CH Equity,NaN
4,4.939849,000002 CH Equity,NaN
5,4.377590,601288 CH Equity,NaN
6,4.019117,000725 CH Equity,NaN
7,3.629410,002415 CH Equity,NaN
8,3.563850,601398 CH Equity,NaN
9,3.439010,601668 CH Equity,NaN


In [5]:
for index in df.index:
    code = df.loc[index, 'wind_code']
    code = code.rstrip(' CH Equity')
    if code.startswith('6'):
        code += '.SH'
    else:
        code += '.SZ'
    df.loc[index, 'wind_code'] = code

In [7]:
df.to_excel('D:/Data/index-component/HSCAI.HI.xlsx', index=False)